In [1]:
import numpy as np, random, operator, pandas as pd, matplotlib.pyplot as plt
import networkx as nx
from parse import read_input_file, write_output_file
from utils import is_valid_solution, calculate_happiness, calculate_stress_for_room
from utils import convert_dictionary
import sys

## Create necessary classes and functions

Create class to handle "cities"

In [2]:
class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((xDis ** 2) + (yDis ** 2))
        return distance
    
    def __repr__(self):
        return "(" + str(self.x) + "," + str(self.y) + ")"

Create a fitness function

In [3]:
class Fitness: #need to add the output for our result
    def __init__(self, route):
        self.route = route
        self.distance = 0
        self.fitness= 0.0
        self.result = {}
    
    def routeDistance(self): #calculate the stress level and happiness here
        for k in range(1, len(self.route)):
            D = {} #student to room
            group_list = [ [] for _ in range(k)]
            for i in range(k):
                D[self.route[i]] = i
                group_list[i].append(self.route[i])
            for i in range(k, len(self.route)):
                cur = self.route[i]
                # print(type(cur_node), cur_node)
                mini = float('inf')
                maxi = -float('inf')
                cur_group = None
                stress_group = None
                stress_out = False
                for group_num, group in enumerate(group_list):
                    happiness = sum(G[cur][other_person]['happiness'] for other_person in group if G.has_edge(cur, other_person))
                    group_copy = list(group)
                    group_copy.append(cur)
                    stress = calculate_stress_for_room(group_copy, G)
                    if stress < mini:
                        mini = stress
                        stress_group = group_num
                    if stress > s/k:
                        continue
                    if happiness > maxi:
                        cur_group = group_num
                        maxi = happiness # we can add tiebreaking here
                if cur_group == None:
                    cur_group = stress_group
                    stress_out = True
                D[cur] = cur_group
                group_list[cur_group].append(cur)
            if is_valid_solution(D, G, s, k):
                cur_val = calculate_happiness(D, G)
                self.result = D
                return cur_val
        return -1
    
    def routeFitness(self):
        if self.fitness == 0:
            self.fitness = float(self.routeDistance())
        return self.fitness

## Create our initial population

Route generator

In [4]:
def createRoute(cityList):
    route = random.sample(cityList, len(cityList))
    return route

Create first "population" (list of routes)

In [5]:
def initialPopulation(popSize, cityList):
    population = []

    for i in range(0, popSize):
        population.append(createRoute(cityList))
    return population

## Create the genetic algorithm

Rank individuals

In [6]:
def rankRoutes(population):
    fitnessResults = {}
    for i in range(0,len(population)):
        fitnessResults[i] = Fitness(population[i]).routeFitness()
    return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)

Create a selection function that will be used to make the list of parent routes

In [7]:
def selection(popRanked, eliteSize):
    selectionResults = []
    df = pd.DataFrame(np.array(popRanked), columns=["Index","Fitness"])
    df['cum_sum'] = df.Fitness.cumsum()
    df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()
    
    for i in range(0, eliteSize):
        selectionResults.append(popRanked[i][0])
    for i in range(0, len(popRanked) - eliteSize):
        pick = 100*random.random()
        for i in range(0, len(popRanked)):
            if pick <= df.iat[i,3]:
                selectionResults.append(popRanked[i][0])
                break
    return selectionResults

Create mating pool

In [8]:
def matingPool(population, selectionResults):
    matingpool = []
    for i in range(0, len(selectionResults)):
        index = selectionResults[i]
        matingpool.append(population[index])
    return matingpool

Create a crossover function for two parents to create one child

In [9]:
def breed(parent1, parent2):
    child = []
    childP1 = []
    childP2 = []
    
    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))
    
    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    for i in range(startGene, endGene):
        childP1.append(parent1[i])
        
    childP2 = [item for item in parent2 if item not in childP1]

    child = childP1 + childP2
    return child

Create function to run crossover over full mating pool

In [10]:
def breedPopulation(matingpool, eliteSize):
    children = []
    length = len(matingpool) - eliteSize
    pool = random.sample(matingpool, len(matingpool))

    for i in range(0,eliteSize):
        children.append(matingpool[i])
    
    for i in range(0, length):
        child = breed(pool[i], pool[len(matingpool)-i-1])
        children.append(child)
    return children

Create function to mutate a single route

In [11]:
def mutate(individual, mutationRate):
    for swapped in range(len(individual)):
        if(random.random() < mutationRate):
            swapWith = int(random.random() * len(individual))
            
            city1 = individual[swapped]
            city2 = individual[swapWith]
            
            individual[swapped] = city2
            individual[swapWith] = city1
    return individual

Create function to run mutation over entire population

In [33]:
def mutatePopulation(population, mutationRate):
    mutatedPop = [population[0]]
    
    for ind in range(1, len(population)):
        mutatedInd = mutate(population[ind], mutationRate)
        mutatedPop.append(mutatedInd)
    return mutatedPop

Put all steps together to create the next generation

In [13]:
def nextGeneration(currentGen, eliteSize, mutationRate):
    popRanked = rankRoutes(currentGen)
    selectionResults = selection(popRanked, eliteSize)
    matingpool = matingPool(currentGen, selectionResults)
    children = breedPopulation(matingpool, eliteSize)
    nextGeneration = mutatePopulation(children, mutationRate)
    return nextGeneration

Final step: create the genetic algorithm

In [35]:
def geneticAlgorithm(population, popSize, eliteSize, mutationRate, generations):
    pop = initialPopulation(popSize, population)
    print("Initial happiness: " + str(rankRoutes(pop)[0][1]))
    happiness = -float('inf')
    best = {}
    for i in range(0, generations):
        print(i)
        elite = rankRoutes(pop)[0]
        ans = Fitness(pop[elite[0]])
        if happiness < ans.routeFitness():
            best = ans.result
            happiness = elite[1]
        print("best so far:" + str(happiness))
        print(elite[1])
        print(ans.result)
        pop = nextGeneration(pop, eliteSize, mutationRate)
    
    print("Final happiness: " + str(happiness))
    print(best)
    return best

# Inputs for Our Algorithm
cityList is the student list.

In [36]:
path = "20.in" #change the input path here
G, s = read_input_file(path)
lst = list(G.nodes)
cityList = lst
random.shuffle(lst)
geneticAlgorithm(population=cityList, popSize=40, eliteSize=4, mutationRate=0.02, generations=200) #adjust hyper parameter

Initial happiness: 1729.2350000000001
0
best so far:1729.2350000000001
1729.2350000000001
{7: 0, 11: 1, 5: 2, 4: 3, 1: 4, 19: 2, 18: 2, 0: 4, 15: 3, 6: 3, 12: 4, 17: 2, 2: 3, 13: 1, 8: 1, 16: 4, 3: 1, 10: 0, 14: 0, 9: 0}
1
best so far:1856.136
1856.136
{14: 0, 9: 1, 1: 2, 19: 3, 4: 4, 10: 0, 11: 1, 16: 0, 12: 4, 6: 4, 5: 3, 8: 1, 15: 0, 13: 1, 17: 3, 18: 3, 0: 4, 3: 2, 2: 2, 7: 2}
2
best so far:1873.785
1873.785
{8: 0, 9: 1, 17: 2, 18: 3, 2: 4, 10: 2, 3: 2, 4: 2, 7: 4, 15: 4, 6: 1, 11: 1, 1: 1, 12: 3, 16: 0, 5: 2, 13: 4, 14: 3, 0: 3, 19: 0}
3
best so far:1916.474
1916.474
{12: 0, 6: 1, 5: 2, 15: 3, 4: 4, 0: 0, 14: 3, 10: 0, 3: 3, 2: 3, 18: 0, 9: 1, 19: 2, 16: 2, 11: 1, 17: 2, 8: 4, 7: 4, 1: 1, 13: 4}
4
best so far:1916.474
1916.474
{12: 0, 6: 1, 5: 2, 15: 3, 4: 4, 0: 0, 14: 3, 10: 0, 3: 3, 2: 3, 18: 0, 9: 1, 19: 2, 16: 2, 11: 1, 17: 2, 8: 4, 7: 4, 1: 1, 13: 4}
5
best so far:1916.474
1916.474
{12: 0, 6: 1, 5: 2, 15: 3, 4: 4, 0: 0, 14: 3, 10: 0, 3: 3, 2: 3, 18: 0, 9: 1, 19: 2, 16: 2, 11:

49
best so far:1983.179
1983.179
{4: 0, 12: 1, 6: 2, 5: 3, 3: 4, 0: 1, 14: 4, 10: 4, 15: 4, 2: 1, 16: 3, 9: 2, 19: 3, 18: 1, 11: 2, 17: 3, 8: 0, 7: 0, 1: 2, 13: 0}
50
best so far:1983.179
1983.179
{4: 0, 12: 1, 6: 2, 5: 3, 3: 4, 0: 1, 14: 4, 10: 4, 15: 4, 2: 1, 16: 3, 9: 2, 19: 3, 18: 1, 11: 2, 17: 3, 8: 0, 7: 0, 1: 2, 13: 0}
51
best so far:1983.179
1983.179
{4: 0, 12: 1, 6: 2, 5: 3, 3: 4, 0: 1, 14: 4, 10: 4, 15: 4, 2: 1, 16: 3, 9: 2, 19: 3, 18: 1, 11: 2, 17: 3, 8: 0, 7: 0, 1: 2, 13: 0}
52
best so far:1983.179
1983.179
{4: 0, 12: 1, 6: 2, 5: 3, 3: 4, 0: 1, 14: 4, 10: 4, 15: 4, 2: 1, 16: 3, 9: 2, 19: 3, 18: 1, 11: 2, 17: 3, 8: 0, 7: 0, 1: 2, 13: 0}
53
best so far:1983.179
1983.179
{4: 0, 12: 1, 6: 2, 5: 3, 3: 4, 0: 1, 14: 4, 10: 4, 15: 4, 2: 1, 16: 3, 9: 2, 19: 3, 18: 1, 11: 2, 17: 3, 8: 0, 7: 0, 1: 2, 13: 0}
54
best so far:1983.179
1983.179
{4: 0, 12: 1, 6: 2, 5: 3, 3: 4, 0: 1, 14: 4, 10: 4, 15: 4, 2: 1, 16: 3, 9: 2, 19: 3, 18: 1, 11: 2, 17: 3, 8: 0, 7: 0, 1: 2, 13: 0}
55
best so far:1

99
best so far:1985.534
1985.534
{4: 0, 12: 1, 8: 2, 5: 3, 3: 4, 0: 1, 14: 4, 10: 4, 15: 4, 2: 1, 16: 3, 9: 2, 7: 2, 11: 0, 17: 3, 1: 3, 6: 0, 13: 0, 19: 0, 18: 1}
100
best so far:1985.534
1985.534
{4: 0, 12: 1, 8: 2, 5: 3, 3: 4, 0: 1, 14: 4, 10: 4, 15: 4, 2: 1, 16: 3, 9: 2, 7: 2, 11: 0, 17: 3, 1: 3, 6: 0, 13: 0, 19: 0, 18: 1}
101
best so far:1985.534
1985.534
{4: 0, 12: 1, 8: 2, 5: 3, 3: 4, 0: 1, 14: 4, 10: 4, 15: 4, 2: 1, 16: 3, 9: 2, 7: 2, 11: 0, 17: 3, 1: 3, 6: 0, 13: 0, 19: 0, 18: 1}
102
best so far:1985.534
1985.534
{4: 0, 12: 1, 8: 2, 5: 3, 3: 4, 0: 1, 14: 4, 10: 4, 15: 4, 2: 1, 16: 3, 9: 2, 7: 2, 11: 0, 17: 3, 1: 3, 6: 0, 13: 0, 19: 0, 18: 1}
103
best so far:1985.534
1985.534
{4: 0, 12: 1, 8: 2, 5: 3, 3: 4, 0: 1, 14: 4, 10: 4, 15: 4, 2: 1, 16: 3, 9: 2, 7: 2, 11: 0, 17: 3, 1: 3, 6: 0, 13: 0, 19: 0, 18: 1}
104
best so far:1985.534
1985.534
{4: 0, 12: 1, 8: 2, 5: 3, 3: 4, 0: 1, 14: 4, 10: 4, 15: 4, 2: 1, 16: 3, 9: 2, 7: 2, 11: 0, 17: 3, 1: 3, 6: 0, 13: 0, 19: 0, 18: 1}
105
best so

149
best so far:1985.534
1985.534
{4: 0, 12: 1, 8: 2, 5: 3, 3: 4, 0: 1, 14: 4, 10: 4, 15: 4, 2: 1, 16: 3, 9: 2, 7: 2, 11: 0, 17: 3, 1: 3, 6: 0, 13: 0, 19: 0, 18: 1}
150
best so far:1985.534
1985.534
{4: 0, 12: 1, 8: 2, 5: 3, 3: 4, 0: 1, 14: 4, 10: 4, 15: 4, 2: 1, 16: 3, 9: 2, 7: 2, 11: 0, 17: 3, 1: 3, 6: 0, 13: 0, 19: 0, 18: 1}
151
best so far:1985.534
1985.534
{4: 0, 12: 1, 8: 2, 5: 3, 3: 4, 0: 1, 14: 4, 10: 4, 15: 4, 2: 1, 16: 3, 9: 2, 7: 2, 11: 0, 17: 3, 1: 3, 6: 0, 13: 0, 19: 0, 18: 1}
152
best so far:1985.534
1985.534
{4: 0, 12: 1, 8: 2, 5: 3, 3: 4, 0: 1, 14: 4, 10: 4, 15: 4, 2: 1, 16: 3, 9: 2, 7: 2, 11: 0, 17: 3, 1: 3, 6: 0, 13: 0, 19: 0, 18: 1}
153
best so far:1985.534
1985.534
{4: 0, 12: 1, 8: 2, 5: 3, 3: 4, 0: 1, 14: 4, 10: 4, 15: 4, 2: 1, 16: 3, 9: 2, 7: 2, 11: 0, 17: 3, 1: 3, 6: 0, 13: 0, 19: 0, 18: 1}
154
best so far:1985.534
1985.534
{4: 0, 12: 1, 8: 2, 5: 3, 3: 4, 0: 1, 14: 4, 10: 4, 15: 4, 2: 1, 16: 3, 9: 2, 7: 2, 11: 0, 17: 3, 1: 3, 6: 0, 13: 0, 19: 0, 18: 1}
155
best s

199
best so far:1985.534
1985.534
{4: 0, 12: 1, 8: 2, 5: 3, 3: 4, 0: 1, 14: 4, 10: 4, 15: 4, 2: 1, 16: 3, 9: 2, 7: 2, 11: 0, 17: 3, 1: 3, 6: 0, 13: 0, 19: 0, 18: 1}
Final happiness: 1985.534
{4: 0, 12: 1, 8: 2, 5: 3, 3: 4, 0: 1, 14: 4, 10: 4, 15: 4, 2: 1, 16: 3, 9: 2, 7: 2, 11: 0, 17: 3, 1: 3, 6: 0, 13: 0, 19: 0, 18: 1}


{4: 0,
 12: 1,
 8: 2,
 5: 3,
 3: 4,
 0: 1,
 14: 4,
 10: 4,
 15: 4,
 2: 1,
 16: 3,
 9: 2,
 7: 2,
 11: 0,
 17: 3,
 1: 3,
 6: 0,
 13: 0,
 19: 0,
 18: 1}

## Running the genetic algorithm

Create list of cities

In [17]:
cityList = []

for i in range(0,25):
    cityList.append(City(x=int(random.random() * 200), y=int(random.random() * 200)))

Run the genetic algorithm

In [18]:
geneticAlgorithm(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=500)

Initial distance: 2249.7240306428266
Final distance: 804.4841773781344


[(163,56),
 (140,97),
 (177,120),
 (187,141),
 (134,172),
 (113,170),
 (69,159),
 (46,194),
 (36,196),
 (14,184),
 (37,176),
 (45,152),
 (50,153),
 (68,116),
 (77,85),
 (29,72),
 (27,64),
 (71,50),
 (54,20),
 (59,3),
 (105,32),
 (135,38),
 (167,14),
 (192,14),
 (174,43)]

## Plot the progress

Note, this will win run a separate GA

In [ ]:
def geneticAlgorithmPlot(population, popSize, eliteSize, mutationRate, generations):
    pop = initialPopulation(popSize, population)
    progress = []
    progress.append(rankRoutes(pop)[0][1])
    
    for i in range(0, generations):
        pop = nextGeneration(pop, eliteSize, mutationRate)
        progress.append(rankRoutes(pop)[0][1])
    
    plt.plot(progress)
    plt.ylabel('Distance')
    plt.xlabel('Generation')
    plt.show()

Run the function with our assumptions to see how distance has improved in each generation

In [ ]:
geneticAlgorithmPlot(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=1000)